In [1]:
import pandas as pd
import matplotlib as plt
%matplotlib inline
from TickTocTest.ticktoctest.masterDB import mergeMaster, updateDB, engine_master, tables
from TickTocTest.ticktoctest.tickToc import getDBdata, db_session, getDF
from Altcoin.altcoin.coinData import CoinData
from Altcoin.altcoin.getData import getData
import requests, json
from datetime import datetime, timedelta

In [2]:
start = datetime.strptime('2018-12-01 00:00:00', '%Y-%m-%d %H:%M:%S')
finish = datetime.strptime('2018-12-02 14:00:00', '%Y-%m-%d %H:%M:%S')
start, finish

(datetime.datetime(2018, 12, 1, 0, 0), datetime.datetime(2018, 12, 2, 14, 0))

In [ ]:
# starts = str(int(start.timestamp())*1000)
# finishs = str(int(finish.timestamp())*1000)
# starts, finishs


Couldn't get bitfinex endpoint to work with ?start=starts?end=finishs. So I grabbed a ?limit=1000 and cropped it

In [ ]:
# url = f'https://api.bitfinex.com/v2/candles/trade:15m:t{sym}USD/hist?limit=1000'

# OOPS
This worked fine for 'OMG' but then put the 'OMG' data into 'trx' table. So now we need to delete some
rows from 'trx' table

#### The Amswer is put the url (line above) inthe for loop below ... Doh


In [ ]:
# with engine_master.begin() as conn:
#     for sym in ['OMG', 'TRX']:
#         url = f'https://api.bitfinex.com/v2/candles/trade:15m:t{sym}USD/hist?limit=1000'
#         res = requests.get(url)
#         data = res.json()
#         df = pd.DataFrame(data, columns=['MTS', 'Open','Close','High','Low','Volume'])
#         df.set_index('MTS', drop=True, inplace=True)
#         df.sort_values(by=['MTS'], inplace=True)
#         df.index = pd.to_datetime(df.index, unit='ms')
#         DF = df.loc['2018-12-01': '2018-12-02 14:0:0'].copy()
        
#         updateDB(DF, tables[sym.lower()], conn)

In [ ]:
# Session = db_session()
# session= Session()
# trx = CoinData('trx', getDBdata, session)
# trx.plot('2018-11-01')

### Delete some rows

In [ ]:
# query = session.query(table.MTS)
# query = query.filter((table.MTS >= start) & (table.MTS <= finish))

In [ ]:
# query.delete()
# session.commit()

In [ ]:
# trx.plot('2018-11-01')

In [ ]:
# session.close()

### Get Bitfinex data

In [ ]:
# with engine_master.begin() as conn:
sym = EDO
url = f'https://api.bitfinex.com/v2/candles/trade:15m:t{sym}USD/hist?limit=1000'
res = requests.get(url)
data = res.json()
df = pd.DataFrame(data, columns=['MTS', 'Open','Close','High','Low','Volume'])
df.set_index('MTS', drop=True, inplace=True)
df.sort_values(by=['MTS'], inplace=True)
df.index = pd.to_datetime(df.index, unit='ms')
DF = df.loc[start: finish].copy()

#     updateDB(DF, tables[sym.lower()], conn)
DF.head(20)

### Get Compare data update missing data


In [3]:
# coins = ['XEM','BNB','BCN','ICX','ONT','ZIL','AE','ZRX']
coins = ['BCH']
with engine_master.begin() as conn:
    for sym in coins:
        compare_url = f'https://min-api.cryptocompare.com/data/histohour?fsym={sym}&tsym=USD&limit=2000&aggregate=1&e=CCCAGG'
        res = requests.get(compare_url)
        data = res.json()
        if data['Type'] >= 100:
            DF = pd.DataFrame(data['Data'][1:])
            DF['MTS'] = pd.to_datetime(DF['time'], unit='s')
            DF['Volume'] = DF['volumefrom'] + DF['volumeto']
            DF.drop(['time', 'volumefrom', 'volumeto'], inplace=True, axis=1)
            DF.set_index('MTS', drop=True, inplace=True)
            DF.rename(columns={'close': 'Close', 'open': 'Open', 'low': 'Low', 'high': 'High'},
                      inplace=True)
            DF = DF[['Open', 'Close', 'High', 'Low', 'Volume']]
        else: 
            print(data['Type'], sym)
            
#         updateDB(DF, tables[sym.lower()], conn)

In [4]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2000 entries, 2018-09-27 06:00:00 to 2018-12-19 13:00:00
Data columns (total 5 columns):
Open      2000 non-null float64
Close     2000 non-null float64
High      2000 non-null float64
Low       2000 non-null float64
Volume    2000 non-null float64
dtypes: float64(5)
memory usage: 93.8 KB


In [5]:
from TickTocTest.ticktoctest.tickToc import getDF
db_name = f'sqlite:///c:\\data\\sqlite\\db\\master_db.db'
Session = db_session(db_name)
session = Session()

In [6]:
# Combine Altcoin with mysql ticktoc15m.db
for coin in ['BCHUSD']:
    sym = coin[:-3]
    df6 = getData(sym, '6h')
    df1 = getData(sym, '1h')
    db_df = getDF(sym.lower(), session)
    
    dfata = pd.concat([df6, df1, db_df, DF])
    dfata.drop_duplicates(inplace=True)
    dfata = DF.groupby('MTS')['Open', 'Close', 'High', 'Low', 'Volume'].mean()
    dfata.sort_values(by=['MTS'])


In [8]:
dfata.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2000 entries, 2018-09-27 06:00:00 to 2018-12-19 13:00:00
Data columns (total 5 columns):
Open      2000 non-null float64
Close     2000 non-null float64
High      2000 non-null float64
Low       2000 non-null float64
Volume    2000 non-null float64
dtypes: float64(5)
memory usage: 93.8 KB
